# Airbnb

We wish to work on airbnb dataset in order to make some data presentations.
The dataset comes from here : https://www.kaggle.com/datasets/joyshil0599/airbnb-listing-data-for-data-science

# Import des packages

In [5]:
 ! pip install opendatasets

In [7]:
import opendatasets as od

In [299]:
import datetime as dt
import numpy as np

# Data Import

In [1013]:
od.download("https://www.kaggle.com/datasets/joyshil0599/airbnb-listing-data-for-data-science/download?datasetVersionNumber=1")
df = pd.read_csv(r".\airbnb-listing-data-for-data-science\airnb.csv")

Skipping, found downloaded files in ".\airbnb-listing-data-for-data-science" (use force=True to force download)


## Data Cleaning

In [1014]:
df.isna().sum()

Title                       0
Detail                      0
Date                        0
Price(in dollar)            0
Offer price(in dollar)    787
Review and rating           0
Number of bed               0
dtype: int64

In [1015]:
df["Review and rating"] = df["Review and rating"].replace("None", np.nan).replace("New", np.nan)
df.isna().sum()

Title                       0
Detail                      0
Date                        0
Price(in dollar)            0
Offer price(in dollar)    787
Review and rating          22
Number of bed               0
dtype: int64

## Data Management

### Dates : start, end and number of days

In [1016]:
df['Start_date'] = df['Date'].str.split("-").str[0].str.strip() + ", 2023"
df['End_date'] = df['Date'].str.split("-").str[1].str.strip()
df['End_date'] = np.where(df['End_date'].str.contains('[a-zA-Z]', regex=True), 
                                df['End_date'] + ", 2023",
                                df['Start_date'].str.split().str[0] + " " + df['End_date'] + ", 2023")
df['Start_date'] = pd.to_datetime(df['Start_date'], format="%b %d, %Y")
df['End_date'] = pd.to_datetime(df['End_date'], format="%b %d, %Y")
df['Number_of_days'] = (df['End_date'] - df['Start_date']) / np.timedelta64(1, 'D')

### Price

In [1017]:
df["Price(in dollar)"] = df["Price(in dollar)"].str.replace(",","")
df["Price(in dollar)"] = df["Price(in dollar)"].astype(float)
df["Offer price(in dollar)"] = df["Offer price(in dollar)"].str.replace(",","")
df["Offer price(in dollar)"] = df["Offer price(in dollar)"].astype(float)

### Reviews and rating 

In [1018]:
df["Rating"] = df["Review and rating"].str.split().str[0].astype(float)
df["Review_number"] = df["Review and rating"].str.split().str[1].str.strip().str.replace("(","", regex = True).str.replace(")","", regex = True)
df["Review_number"] = df["Review_number"].replace(np.nan, "0")
df["Review_number"] = df["Review_number"].astype(int)

### Bedding

In [1019]:
df["Type_of_beddings"] = df["Number of bed"].str.split().str[1].str.strip().astype(str)
df["Number_of_beddings"] = df["Number of bed"].str.split().str[0].str.strip().astype(int)
df["Number_of_beddings"].value_counts()

1     369
2     325
3     131
4      75
5      19
6      17
9       5
8       4
7       3
11      1
22      1
17      1
13      1
10      1
Name: Number_of_beddings, dtype: int64

In [1020]:
# We must group beds in similar categories then accord each a capacity and multiply it with the number of beds
df["Type_of_beddings"].unique()
df["Type_of_beddings"].value_counts()

beds      452
queen     163
king      110
bed       100
double     91
sofa       23
single     11
bunk        3
Name: Type_of_beddings, dtype: int64

In [1021]:
bed_group_dict ={'beds':1,
 'queen':2, 
 'double':2, 
 'king':2,
 'bed':1,
 'sofa':1, 
 'single':1,
 'bunk':2}

df["Size_beddings"] = df["Type_of_beddings"].replace(bed_group_dict).astype(int)
df["Hosting_capacity"] = df["Size_beddings"] * df["Number_of_beddings"]
df["Price_per_bedding"] = df["Price(in dollar)"] / df["Number_of_beddings"]

In [1022]:
df["Number_of_beddings_grouped"] = np.where(df["Number_of_beddings"] >= 5,
                                   "5 and more",
                                   df["Number_of_beddings"].astype(str))

### Type of accomodation

In [1023]:
df["Accomodation_type"] = df["Title"].str.split(" in ").str[0].str.strip()

In [1024]:
df["Accomodation_type"].value_counts()

Home                  123
Apartment             119
Cabin                 116
Condo                  95
Room                   83
Villa                  62
Tiny home              42
Treehouse              37
Cottage                34
Guesthouse             26
Place to stay          24
Loft                   19
Hut                    17
Farm stay              16
Guest suite            13
Dome                   11
Bungalow               11
Hotel                   9
Chalet                  9
Hotel room              6
Shipping container      6
Earthen home            6
Vacation home           5
Resort                  5
Nature lodge            5
Tent                    5
Barn                    5
Camper/RV               4
Yurt                    4
Shepherd’s hut          4
Trullo                  4
Boat                    4
Boutique hotel          4
Townhouse               4
Campsite                3
Castle                  3
Cave                    2
Houseboat               2
Lighthouse  

### Area

In [1025]:
df["Place"] = df["Title"].str.split(" in ").str[1].str.strip()
df["City"] = df["Place"].str.split(", ").str[0].str.strip()
df["Country"] = np.where(df["Place"].str.count(", ")==0,
                         "US",
                         df["Place"].str.split(", ").str[-1].str.strip())
df["US_State"] = np.where(df["Place"].str.count(", ")==2,
                           df["Place"].str.split(", ").str[1],
                           "")

In [1026]:
df["Country"].unique()

array(['US', 'Mexico', 'Canada', 'Costa Rica', 'Cuba', 'Colombia',
       'Jamaica', 'Dominican Republic', 'Puerto Rico', 'El Salvador',
       'Curaçao', 'U.S. Virgin Islands', 'Norway', 'Poland', 'Turkey',
       'Iceland', 'Czechia', 'Italy', 'Spain', 'Sweden', 'Portugal', 'UK',
       'Germany', 'France', 'Greece', 'Switzerland', 'Ireland', 'Tunisia',
       'Montenegro', 'Austria', 'Indonesia', 'Philippines', 'Thailand',
       'Vietnam', 'Malaysia', 'Taiwan'], dtype=object)

### Amenities

In [1027]:
# Swimming pool
pool_list = ['pool', 'Pool']
df['Pool'] = df['Detail'].str.contains('|'.join(pool_list)).astype(int)
# Sauna
sauna_list = ['sauna', 'Sauna', 'hottube']
df['Sauna'] = df['Detail'].str.contains('|'.join(sauna_list)).astype(int)

### Get localisations

In [945]:
import requests
from unidecode import unidecode
import urllib.request, json
import time

In [952]:
df.shape

(953, 20)

In [968]:
df_places = df[['City','Country']].drop_duplicates()
df_places = df_places.reset_index(drop=True)
df_places['Index'] = df_places.index

In [954]:
df2.shape

(594, 2)

In [971]:
Geoloc_list = []
Geloc_rejected = []
for i in range(len(df2)):
    time.sleep(2)
    url = unidecode('https://nominatim.openstreetmap.org/search?q={},{}&format=json'.format(df_places['City'][i], df_places['Country'][i]))
    url = url.replace(" ", "%20")
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    if len(data) == 0:
        Geloc_rejected.append([i, url])
        pass
    else:
        Geoloc_list.append([i, data[0]['lat'], data[0]['lon']])

In [996]:
len(Geoloc_dict), len(Geloc_rejected)

(562, 32)

In [976]:
df_geoloc = pd.DataFrame(Geoloc_dict, columns = ['Index', 'Latitude', 'Longitude'])

In [1008]:
df_locations = pd.merge(df_places, df_geoloc, on=['Index']).drop(columns=['Index'])

In [1028]:
df = pd.merge(df, df_locations, on=['City','Country'], how="left")

In [1031]:
df.shape

(953, 27)

In [1029]:
df.dtypes

Title                                 object
Detail                                object
Date                                  object
Price(in dollar)                     float64
Offer price(in dollar)               float64
Review and rating                     object
Number of bed                         object
Start_date                    datetime64[ns]
End_date                      datetime64[ns]
Number_of_days                       float64
Rating                               float64
Review_number                          int32
Type_of_beddings                      object
Number_of_beddings                     int32
Size_beddings                          int32
Hosting_capacity                       int32
Price_per_bedding                    float64
Number_of_beddings_grouped            object
Accomodation_type                     object
Place                                 object
City                                  object
Country                               object
US_State  

### Column Selection and Output

In [1032]:
df.columns.tolist()

['Title',
 'Detail',
 'Date',
 'Price(in dollar)',
 'Offer price(in dollar)',
 'Review and rating',
 'Number of bed',
 'Start_date',
 'End_date',
 'Number_of_days',
 'Rating',
 'Review_number',
 'Type_of_beddings',
 'Number_of_beddings',
 'Size_beddings',
 'Hosting_capacity',
 'Price_per_bedding',
 'Number_of_beddings_grouped',
 'Accomodation_type',
 'Place',
 'City',
 'Country',
 'US_State',
 'Pool',
 'Sauna',
 'Latitude',
 'Longitude']

In [1033]:
columns = ['Detail',
 'Price(in dollar)',
 'Offer price(in dollar)',
 'Price_per_bedding',
 'Start_date',
 'End_date',
 'Number_of_days',
 'Rating',
 'Review_number',
 'Type_of_beddings',
 'Size_beddings',
 'Hosting_capacity',
 'Number_of_beddings',
 'Number_of_beddings_grouped',
 'Accomodation_type',
 'City',
 'Country',
 'US_State',
 'Pool',
 'Sauna',
 'Latitude',
 'Longitude']
df = df[columns]
df.to_csv("data_airbnb.csv", sep=";", index=False)